Memory: `SystemMessage` + `history` + `HumanMessage` + `agent_scratchpad`


### 1. History


1. `ConversationBufferMemory`


- context: `(HummanMessage, AIMessage)`


In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# from langchain_openai import OpenAI
from core.llm import CHAT_LLM as llm

# llm = OpenAI(temperature=0)

# Here it is by default set to "AI"
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")
conversation.predict(input="What's the weather?")

/home/vscode/.cache/pypoetry/virtualenvs/langchain-tutorial-9TtSrW0h-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today?
Human: What's the weather?
AI:

> Finished chain.


'The current weather in your location is 75 degrees Fahrenheit with partly cloudy skies. There is a 20% chance of rain later this evening.'

- context: `(HummanMessage, AIMessage, FunctionMessage)`


In [2]:
import json

from langchain_core.messages import HumanMessage, AIMessage, FunctionMessage
from langchain_core.agents import AgentActionMessageLog
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.messages import SystemMessage
from langchain.agents.format_scratchpad import format_to_openai_function_messages


def generate_artificial_tool_response(tool: str, tool_input: dict):
    """Generate artificial tool response.

    Args:
        tool (str): The tool.
        tool_input (dict): The tool input.

    Returns:
        dict: The artificial tool response.
    """
    return AgentActionMessageLog(
        tool,
        tool_input,
        log=f"\nInvoking: `{tool}` with `{tool_input}`\n\n\n",
        message_log=[
            AIMessage(
                content="",
                additional_kwargs={
                    "function_call": {
                        "arguments": json.dumps(tool_input, ensure_ascii=False),
                        "name": tool,
                    }
                },
            )
        ],
    )


memory = ConversationBufferMemory(return_messages=True)

agent_action = generate_artificial_tool_response(
    tool="greeting",
    tool_input={"time": "morning", "location": "Tokyo"},
)
observation = "안녕하세요!"

memory.chat_memory.add_messages(
    [
        HumanMessage(content="안녕~"),
        *format_to_openai_function_messages([(agent_action, observation)]),
    ]
)

preprocess = RunnablePassthrough.assign(
    history=lambda input: memory.load_memory_variables({})["history"]
)
template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are function calling assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
template_with_preprocess = preprocess | template
val = template_with_preprocess.invoke({"input": "그래~"})
print(val.to_string())

System: You are function calling assistant
Human: 안녕~
AI: {'arguments': '{"time": "morning", "location": "Tokyo"}', 'name': 'greeting'}
Function: 안녕하세요!
Human: 그래~


### 2. Agent scratchpad


In [3]:
import json

from langchain_core.messages import AIMessage
from langchain_core.agents import AgentActionMessageLog
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.messages import SystemMessage
from langchain.agents.format_scratchpad import format_to_openai_function_messages


def generate_artificial_tool_response(tool: str, tool_input: dict):
    """Generate artificial tool response.

    Args:
        tool (str): The tool.
        tool_input (dict): The tool input.

    Returns:
        dict: The artificial tool response.
    """
    return AgentActionMessageLog(
        tool,
        tool_input,
        log=f"\nInvoking: `{tool}` with `{tool_input}`\n\n\n",
        message_log=[
            AIMessage(
                content="",
                additional_kwargs={
                    "function_call": {
                        "arguments": json.dumps(tool_input, ensure_ascii=False),
                        "name": tool,
                    }
                },
            )
        ],
    )


memory = ConversationBufferMemory(return_messages=True)

agent_action = generate_artificial_tool_response(
    tool="greeting",
    tool_input={"time": "morning", "location": "Tokyo"},
)
observation = "안녕하세요!"
memory.chat_memory.add_messages(
    [
        HumanMessage(content="안녕~"),
        *format_to_openai_function_messages([(agent_action, observation)]),
    ]
)
memory.chat_memory.add_messages(
    [
        HumanMessage(content="안녕~"),
        AIMessage(content="어.. 또..?"),
    ]
)

agent_action = generate_artificial_tool_response(
    tool="request_again",
    tool_input={"utterance": "지나친 세일은 게임의 가치를 낮출 뿐입니다"},
)
observation = "75% 세일을 하지 않을 것입니다"


preprocess = RunnablePassthrough.assign(
    history=lambda input: memory.load_memory_variables({})["history"],
    agent_scratchpad=lambda input: format_to_openai_function_messages(
        input["intermediate_steps"]
    ),
)
template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are function calling assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
template_with_preprocess = preprocess | template
val = template_with_preprocess.invoke(
    {"input": "해봐! 빨리!", "intermediate_steps": [(agent_action, observation)]}
)
print(val.to_string())

System: You are function calling assistant
Human: 안녕~
AI: {'arguments': '{"time": "morning", "location": "Tokyo"}', 'name': 'greeting'}
Function: 안녕하세요!
Human: 안녕~
AI: 어.. 또..?
Human: 해봐! 빨리!
AI: {'arguments': '{"utterance": "지나친 세일은 게임의 가치를 낮출 뿐입니다"}', 'name': 'request_again'}
Function: 75% 세일을 하지 않을 것입니다


In [4]:
import json

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.agents import AgentActionMessageLog
from langchain.memory import ConversationBufferMemory
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

# from langchain_openai import OpenAI
from core.llm import CHAT_LLM as llm


def generate_artificial_tool_response(tool: str, tool_input: dict):
    """Generate artificial tool response.

    Args:
        tool (str): The tool.
        tool_input (dict): The tool input.

    Returns:
        dict: The artificial tool response.
    """
    return AgentActionMessageLog(
        tool,
        tool_input,
        log=f"\nInvoking: `{tool}` with `{tool_input}`\n\n\n",
        message_log=[
            AIMessage(
                content="",
                additional_kwargs={
                    "function_call": {
                        "arguments": json.dumps(tool_input, ensure_ascii=False),
                        "name": tool,
                    }
                },
            )
        ],
    )


memory = ConversationBufferMemory(return_messages=True)

agent_action = generate_artificial_tool_response(
    tool="greeting",
    tool_input={"time": "morning", "location": "위례신도시"},
)
observation = "안녕하세요!"
memory.chat_memory.add_messages(
    [
        HumanMessage(content="안녕~"),
        *format_to_openai_function_messages([(agent_action, observation)]),
    ]
)


template = """You are AI assistant extracting the location and time information from the user input.

History:
{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)


# llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, prompt=prompt, verbose=True, memory=memory)
conversation.invoke({"input": "나 어디 사는지 알아?"})



> Entering new ConversationChain chain...
Prompt after formatting:
You are AI assistant extracting the location and time information from the user input.

History:
[HumanMessage(content='안녕~'), AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"time": "morning", "location": "위례신도시"}', 'name': 'greeting'}}), FunctionMessage(content='안녕하세요!', name='greeting')]

Conversation:
Human: 나 어디 사는지 알아?
AI:

> Finished chain.


{'input': '나 어디 사는지 알아?',
 'history': [HumanMessage(content='안녕~'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"time": "morning", "location": "위례신도시"}', 'name': 'greeting'}}),
  FunctionMessage(content='안녕하세요!', name='greeting'),
  HumanMessage(content='나 어디 사는지 알아?'),
  AIMessage(content='네, 어디 사시는지 알려주세요.')],
 'response': '네, 어디 사시는지 알려주세요.'}